# 1. <a id='toc1_'></a>[QISKIT Lab 6 - VQE H2 and LiH](#toc0_)
 1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
1. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
1. **P. Djorwe**, djorwepp@gmail.com
    * Department of Physics, Faculty of Science, University of Ngaoundere
       
May 2023

$
\newcommand{\ad}{a^\dagger} % Operateur bosonique adjoint 
\newcommand{\mt}[1]{\mathtt{#1}} %  Use mathtt
\newcommand{\mel}[3]{\langle #1|#2|#3\rangle} %Matrix element
\newcommand{\ket}[1]{|#1\rangle}
$

L'algorithme VQE (Variational Quantum Eigensolver), dans la contexte de la chimie quantique ou de la modélisation moléculaire, utilise la méthode variationelle, et deux processeurs, quantique et classique, pour déterminer l'énergie la plus basse associée à la valeur propre de l'etat fondamental ou des états excités.

## 1.1. <a id='toc1_1_'></a>[Algorithme détaillé de la VQE](#toc0_)
On peut résumer cet algorithme en deux grandes parties qu'illustre la figure ci-dessous.

<center>
<img src="./Graphics/VQE_Diagram.png" width=700 />
</center>
                                                          

### 1.1.1. <a id='toc1_1_1_'></a>[Processeur quantique](#toc0_)

Le processeur quantique comporte trois étapes fondamentales :

1. Définir le circuit quantique ou porte quantique $\mathtt{U}(\vec{\theta})$;
2. Préparer de la fonction d'essai paramétré $|\Psi (\vec{\theta})\rangle$ appelée **Ansatz**, qui est essentiellement une estimation de l'état fondamental, à cet effet, on choisit arbitrairement un état de référence $|\psi_0\rangle$ sur lequel on applique $\mathtt{U}(\vec{\theta})$,
	\begin{equation}
		|\Psi (\vec{\theta})\rangle= \mathtt{U}(\vec{\theta})|\psi_0 \rangle=\sum_i\alpha_i|E_i\rangle.
	\end{equation}
3. Mesurer de la valeur moyenne ou fonction de coût
\begin{equation}
C(\vec\theta)=\langle\Psi(\vec\theta)|\mathtt{H}|\Psi(\vec\theta)\rangle
=\langle \psi_0| \mathtt{U}^\dagger (\vec{\theta})\mathtt{HU} (\vec{\theta})|\psi_0\rangle.
\end{equation}
Selon la décomposition spectrale, $\mathtt{H}$ peut être représenté par:
\begin{equation}
\mathtt{H}=\sum_i E_i|E_i\rangle\langle E_i|.
\end{equation}
En vertu du [théorème variationnel](https://en.wikipedia.org/wiki/Vcorrespondariational_method_(quantum_mechanics)) de Rayleigh-Ritz, la valeur moyenne est toujours supérieure ou égale à la valeur propre $E_0$ la plus basse de l'Hamiltonien $\mathtt{H}$, qui correspond à l'état fondamental $|E_{\min}\rangle$:
\begin{equation}
C(\vec\theta)=\langle \psi_0| \mathtt{U}^\dagger (\vec{\theta})\mathtt{HU} (\vec{\theta})|\psi_0\rangle
=\sum_i|\alpha_i|^2E_i\geq E_{\min}.
\end{equation}
Le problème se résume à trouver un tel choix optimal de paramètres $\vec\theta=(\theta_1,\dots,\theta_n)^T$ à valeurs réelles, permettant de trouver la valeur moyenne minimale $E_{\min}$ qui est l'énergie de l'état fondamental et l'état correspondant est l'état fondamental $|E_{\min}\rangle$.


### 1.1.2. <a id='toc1_1_2_'></a>[Processeur classique](#toc0_)

Grâce au processeur quantique, on obtient une valeur moyenne dépendante des paramètres. Cette valeur peut être minimisée avec une méthode d'optimisation qui permet d'ajuster les paramètres de l'état d'essai. L'algorithme procède alors de façon itérative, l'optimiseur classique proposant de nouvelles valeurs de paramètres pour l'état d'essai.

En gros, dans le processeur classique :
1. Minimiser la valeur moyenne ou fonction de coût $C(\vec\theta)$ en faisant varier les paramètres $\vec{\theta}$ de l'_Ansatz_, en utilisant un optimiseur classique.
2. Itèrer jusqu'à ce que le critère de convergence ($10^{-7}$) soit atteint et que $|\psi(\vec{\theta})\rangle\simeq |E_0(\vec{\theta})\rangle$.


### 1.1.3. <a id='toc1_1_3_'></a>[Défis ou challenges de la VQE](#toc0_)

1. La taille de l'espace de Hilbert en fonction du nombre des opérations de portes nécessaires ou profondeur du circuit pour obtenir le précisions souhaitée pour les valeurs attendues (comme l'énergie du système).

2. Le nombre de portes parametrés (nombre de paramètres à optimiser) en fonction de l'amplitude des gradients pour variables de circuits.

3. La taille de l'espace de Hilbert en fonction du nombre de mesures nécessaire pour parvenir à la convergence des propriétés physiques (par exemple, l'énergie du système).


Dans ce qui suit, nous utilisons les **Unitary Coupled Clusters Singles and Doubles** (UCCSD) comme point de départ pour déterminer une fonction d'état d'essai pour la méthode variationnelle, car il est essentiel que l'ansatz VQE soit proche de l'état fondamental réel pour que les calculs VQE réussissent. Dans ce tutoriel, nous nous concentrons sur le calcul de l'état fondamental et de la surface d'énergie potentielle de Born-Oppenheimer (BOPES) pour les molécules d'hydrogène (H2) et d'hydrure de lithium (M) et une macromolécule.

### 1.1.4. <a id='toc1_1_4_'></a>[Unitary Coupled Cluster (UCC) *Ansatz*](#toc0_)

En chimie quantique, la méthode **Unitary Coupled Cluster (UCC)** (de [cluster à couplage unitaire](https://fr.wikipedia.org/wiki/M%C3%A9thode_du_cluster_coupl%C3%A9)) apparait comme une extension de la méthode *Coupled Cluster (CC)* qui est une méthode de traitement de la corrélation électronique. Elle est basée sur l'expression de la fonction d'état à $N$ électrons comme une combinaison linéaire de déterminants de Slater incluant la fonction d'état HF de l'état fondamental et toutes les excitations possibles des orbitales occupées vers des orbitales inoccupées. Ainsi, il sera possible de générer un *Unitary Coupled Cluster Ansatz* en appliquant à un état de référence $\ket{\psi_0}$ un opérateur unitaire qui est une somme anti-Hermitienne d'opérateurs d'excitation et de désexcitation sous la forme $e^{\mathtt{T}(\theta)-\mathtt{T}^\dagger(\theta)}$, qui est un opérateur unitaire
$$
\ket{\psi(\theta)} = \mt{U}(\theta) \ket{\psi_0}
= e^{\mathtt{T}(\theta)-\mathtt{T}^\dagger(\theta)} \ket{\psi_0(\theta)} ,
$$
* $\theta$ est l'amplitude CC. Il représente aussi le paramètre d'optimisation pouvant prendre des valeurs réelles ou imaginaires. Mais les paramètres réels se sont révélés plus précises et plus réalisables ;

* $\mathtt{T}(\theta)$ est l'opérateur de *Cluster* ou opérateur d'excitation complète, défini comme  $\mathtt{T}(\theta)=\sum_{k=1}^N\mathtt{T}_k(\theta)$ avec $\mathtt{T}_k(\theta)$ l'opérateur d'excitation au $k$-ième ordre, qui contient des termes $k$-corps. Par exemple,
    * l'opérateur 
\begin{equation*}
\mathtt{T}_1 = \underset{i\in\rm{unocc}}{\sum_{j\in\rm{occ}}}\theta_{ij}\ad_i a_j,
\end{equation*}  
engendre les **excitations simples** $j\rightarrow i$ (transforme le déterminant HF de référence en une combinaison linéaire des déterminants monoexcités),
    * l'opérateur
 \begin{equation*}
\mathtt{T}_2 = \underset{l>k\in\rm{occ}}{\sum_{i>j\in\rm{unocc}}}\theta_{ijkl} 
\ad_i\ad_j a_k a_l,\ \dots,
\end{equation*}  
engendre les **doubles excitations** (transforme le déterminant HF de référence en une combinaison linéaire des déterminants doublement excités).
    * Les termes d'ordre supérieur (triple, quadruple, etc.) sont possibles, mais sont actuellement rarement pris en charge par les bibliothèques de chimie quantique.
    * "occ" et "unocc" sont définis comme les sites occupés et les sites inoccupés dans l'état de référence.
    * Les opérateurs $\ad_i$ et $a_i$ dans les termes de clusters couplés ci-dessus sont écrits dans une forme canonique, dans laquelle chaque terme est en ordre normal (opérateurs de création sont à gauche de tous les opérateurs d'annihilation).

# 2. <a id='toc2_'></a>[Choix des optimiseurs classiques](#toc0_)

Une fois que l'_Ansatz_ a été sélectionné, ses paramètres doivent être optimisés pour minimiser la valeur attendue de l'Hamiltonien cible. Le processus d'optimisation des paramètres présente divers défis. Par exemple, le matériel quantique a divers types de bruits et donc l'évaluation de la fonction objective (calcul de l'énergie) peut ne pas nécessairement refléter la véritable fonction objectif. De plus, certains optimiseurs effectuent un certain nombre d'évaluations de fonctions objectives en fonction de la cardinalité de l'ensemble de paramètres. __Un optimiseur approprié doit être sélectionné en tenant compte des exigences d'une application__.

- Une stratégie d'optimisation populaire est la __descente de gradient__ (`qiskit.algorithms.optimizers.GradientDescent`) où chaque paramètre est mis à jour dans la direction produisant le plus grand changement local d'énergie. Par conséquent, le nombre d'évaluations effectuées dépend du nombre de paramètres d'optimisation présents. Cela permet à l'algorithme de trouver rapidement un optimum local dans l'espace de recherche. Cependant, cette stratégie d'optimisation reste souvent bloquée à des optima locaux médiocres et est relativement coûteuse en termes de nombre d'évaluations de circuits effectuées. Bien qu'il s'agisse d'une stratégie d'optimisation intuitive, il n'est pas recommandé de l'utiliser dans VQE dans sa forme _brute_ (__barren plateau problem__).
 
  <center><img src="Graphics/descente-de-gradient.png" width="400"/>
  <img src="Graphics/spsa_mntn.png" width="220"/></center> 

- Un optimiseur approprié pour optimiser une fonction objectif bruyante est l'optimiseur d'__approximation stochastique de perturbations simultanées (Simultaneous Perturbation Stochastic Approximation, SPSA),__ (`qiskit.algorithms.optimizers.SPSA`). SPSA se rapproche du gradient de la fonction objectif avec seulement deux mesures. Il le fait en perturbant simultanément tous les paramètres de manière aléatoire, contrairement à la descente de gradient où chaque paramètre est perturbé indépendamment. __Lors de l'utilisation de VQE dans un simulateur bruyant (QasmSimulator) ou sur du matériel réel, SPSA est recommandé comme optimiseur classique__.
    - L'optimiseur Quantum Natural SPSA, QN-SPSA (`qiskit.algorithms.optimizers.QnSPSA`) basé sur SPSA tente d'améliorer la convergence en échantillonnant le gradient naturel au lieu du gradient de premier ordre. Par rapport aux gradients naturels, qui nécessitent $\mathcal{O}(d^2)$ évaluations des valeurs moyennes pour un circuit avec $d$ paramètres, QN-SPSA ne nécessite que $\mathcal{O}(1)$ et peut donc accélérer considérablement le calcul du gradient naturel en sacrifiant une certaine précision. Par rapport à SPSA, QN-SPSA nécessite 4 évaluations supplémentaires de la fidélité.

- Lorsque le bruit n'est pas présent dans l'évaluation de la fonction de coût, comme lors de l'utilisation de VQE avec un simulateur de vecteur d'état (_StatevectorSimulator_), une grande variété d'optimiseurs classiques peut être utile. Par exemple,  
    - l'optimiseur __Constrained Optimization by Linear Approximation, COBYLA,__ (`qiskit.algorithms.optimizers.COBYLA`) qui n'effectue qu'une seule évaluation de fonction objectif par itération d'optimisation, et donc le nombre d'évaluations est indépendant de la cardinalité de l'ensemble de paramètres. Par conséquent, si la fonction objectif est sans bruit et qu'il est souhaitable de minimiser le nombre d'évaluations effectuées, il est recommandé d'essayer COBYLA;
    - l'optimiseur __Programmation des moindres carrés séquentiels (Sequential Least Squares Programming, SLSQP)__ (`qiskit.algorithms.optimizers.SLSQP`), est idéal pour les problèmes mathématiques pour lesquels la fonction objectif et les contraintes sont deux fois continûment différentiables;
    - l'optimiseur __Limited-memory Broyden-Fletcher-Goldfarb-Shanno Bound, L_BFGS_B,__ (`qiskit.algorithms.optimizers.L_BFGS_B`), est une méthode quasi-Newton qui commence par une estimation initiale de la valeur optimale et procède de manière itérative pour affiner cette estimation avec une séquence de meilleures estimations.

### 2.1.1. <a id='toc2_1_1_'></a>[Workflow de la VQE](#toc0_)

<center><img src="Graphics/VQE_Flowchart.jpeg" width="1000"/></center>


1. Renseigner la structure de la molécule.

2. Effectuer le calcul HF dans la base chimique indiquée. Il s'agit en réalité représenter le problème de l'equation de Schrödinguer électronique, $\mathtt{H}_{\rm el}|\Psi\rangle = E_{\rm el}|\Psi\rangle$. $\mathtt{H}_{\rm el}$ est l'Hamiltonien de la classe `qiskit_nature.second_q.hamiltonians.ElectronicEnergy`

3. Extraire, à l'aide du calcul HF précédent, les éléments de matrice 1-integrals $h_{pq}$ et 2-integrals $h_{pqrs}$ qui relient l'Hamiltonien de la seconde quantification 1a celui de la première quantification. Les utiliser pour construire l'Hamiltonien fermionique de la seconde quantification
\begin{equation}
\mathtt{H} = h_0+\sum_{p,q=1}^M h_{pq}\ad_p a_q + \frac12\sum_{p,q,r,s=1}^M h_{pqrs}\ad_p \ad_q a_ra_s ,
\end{equation} 
que `qiskit_nature.second_q.mappers.QubitConverter` converti, grâce à un encodage approprié (JWT, PT ou BKT) en Hamiltonien qubit.

   * Exploiter les symmétries
   \begin{align}
   &[\mathtt{H},\mathtt{N_\uparrow}] = [\mathtt{H},\mathtt{N_\downarrow}] = 0, 
   && \mathtt{N_\uparrow} = \sum_{p=1}^{M/2} \alpha^\dagger_p \alpha_p,
   & \mathtt{N_\downarrow} = \sum_{p=M/2+1}^M \alpha^\dagger_p \alpha_p,
   \end{align}
   pour effectuer la **reduction 2-qubit** (une pour chaque symétrie $\mathbb{Z}$ de l'Hamiltionien) sans modifier la partie inférieure du spectre d'énergie (y compris l'état fondamental). $\mathtt{N_\downarrow}$ et $\mathtt{N_\uparrow}$ sont les opérateurs nombre de particules de spin down et up.

   * Utiliser `qiskit_nature.second_q.transformers.FreezeCoreTransformer` afin d'appliquer l'approximation du noyau gelé (**frozen-core approximation**) pour réduire le nombre possible des excitations uniques ou double et le nombre de qubits.

4. La fonction d'état d'essai ou *Ansatz* $\ket{\psi(\theta)}$ est généré à partir de l'état HF $\ket{\Phi_0}$ en appliquant les opérateurs de clusters q-UCC ou autre forme d'*Ansatz* variationnel (`qiskit_nature.second_q.circuit.library.UCCSD`).

5. Evaluer l'énergie du système $\mel{\psi(\theta)}{\mathtt{H}}{\psi(\theta)}$ en utilisant
   * la primitive `qiskit.primitives.Estimator`;
   * l'optimisation des paramètres du circuit (grâce à `qiskit.algorithms.optimizers`), suivi par la séquence de mesure des propriétés physiques. Ceci peut se faire avec ou sans les mesures du bruit.

6. Repeter les étapes (4) et (5) jusqu'à la convergence, en utilisant un optimiseur classique. Par usage, on fixe le critère de convergence à $10^{-7}$. En effet, il est essentiel que l'*Ansatz* VQE soit proche de l'état fondamental réel pour que les calculs VQE réussissent. Pour obtenir une estimation d'énergie précise de 1 milli-Hartree (mHA), l'*Ansatz* pour le VQE doit être proche du véritable état fondamental de moins d'un sur un million.

<center><img src="Graphics/Qiskit_nature_Flow.png" width="700"/></center>


## 2.2. <a id='toc2_2_'></a>[Molécule H2](#toc0_)

Ici, nous calculons en fait uniquement la partie électronique. Lors de l'utilisation du package Qiskit Nature, l'énergie de répulsion nucléaire sera ajoutée automatiquement pour obtenir l'énergie totale de l'état fondamental.

### 2.2.1. <a id='toc2_2_1_'></a>[Structure électronique](#toc0_)

Nous allons renseigner les données de la molécule et effectuer la conversion en créant un Hamiltonien fermionique (problème électronique précisement) qui sera ensuite converti par la transformation de Jordan Wigner, à un Hamiltonien qubit prêt pour le calcul quantique. 

-  On commence par définir la géométrie de la molécule d'hydrogène.

In [164]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

In [165]:
H2_driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Electronic structure problem
H2_problem = H2_driver.run()
transformer = FreezeCoreTransformer()
H2_problem = transformer.transform(H2_problem)

### 2.2.2. <a id='toc2_2_2_'></a>[Solveur](#toc0_)

Nous devons définir un solveur. Le solveur est l'algorithme par lequel l'état fondamental est calculé.

- Commençons par un exemple purement classique: le `qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver`. Cet algorithme diagonise exactement l'Hamiltonien. Bien qu'il ne soit pas très performant, il peut être utilisé sur de petits systèmes pour vérifier les résultats des algorithmes quantiques.

In [166]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

- Pour définir le solveur VQE, il faut trois éléments essentiels :
  1. Une **forme variationnelle ou *ansatz***. Nous utilisons ici l'*ansatz* Unitary Coupled Cluster (UCC). La valeur par défaut est d'utiliser toutes les excitations simples et doubles. Cependant, le type d'excitation (S, D, SD) ainsi que d'autres paramètres peuvent être sélectionnés. La forme variationnelle `qiskit_nature.second_q.circuit.library.UCCSD` est préparée avec un état initial `qiskit_nature.second_q.circuit.library.HartreeFock`, qui initialise l'occupation de nos qubits en fonction du problème que nous essayons de résoudre.
  2. Une primitive, qui sera ici `qiskit.primitives.Estimator` pour le calcul des valeurs moyennes;
  3. Un optimiseur classique (`qiskit.algorithms.optimizer`).

- Configurons l'*ansatz* et plus précisement, le circuit quantique paramétré, du VQE

In [167]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import (
    ParityMapper, 
    JordanWignerMapper,
    BravyiKitaevMapper
)
from qiskit_nature.second_q.mappers import TaperedQubitMapper

def ansatz(problem,mapper):
    """Ansatz function

    Args:
        mapper 

    Returns: The anstatz
    """    
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    return ansatz

- Configurons la primitive `qiskit.primitives.Estimator` pour le VQE. Elle calcule les valeurs moyennes des circuits d'entrée et des propriétés physiques.

In [168]:
from qiskit.primitives import Estimator

estimator = Estimator()

- Configurons l'optimisseur classique pour le VQE. Pour des raisons de comparabilité, nous allons configurer trois optimisseurs classiques.

In [169]:
from qiskit.algorithms.optimizers import COBYLA, SLSQP, L_BFGS_B

- Assemblons ces composants de l'algorithme VQE dans `qiskit.algorithms.minimum_eigensolvers.VQE` et initialisons le solver VQE.

In [170]:
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

def vqe_solver(problem, mapper, optimizer):
    """ Setup VQE solver

    Args:
        problem : Electronic Structure Problem
        mapper : qubit mapper 
        optimizer : optimizer

    Returns: vqe solver
    """    
    vqe_solver = VQE(estimator, ansatz(problem, mapper), optimizer)

    vqe_solver.initial_point = np.zeros(ansatz(problem,mapper).num_parameters)
    return vqe_solver

### 2.2.3. <a id='toc2_2_3_'></a>[Calcul et résultats](#toc0_)

 - Préparons le solveur d'état fondamental et exécutons-le pour calculer l'état fondamental de la molécule grâce à la classe `qiskit_nature.second_q.algorithms.GroundStateEigensolver`. Il s'agit d'envelopper notre `qiskit_nature.second_q.mappers` et notre algorithme quantique `qiskit.algorithms.minimum_eigensolvers.VQE` (ou classique `qiskit_nature.second_q.algorithms.NumPyMinimumEigensolver`) dans un seul `GroundStateEigensolver`. Pour des raisons de commodité définissons la fonction `run_vqe()`.

Le workflow interne est le suivant :
1. générer les opérateurs de seconde quantification stockés dans notre `problem`" (electronic structure problem);
2. mapper (et potentiellement réduire) les opérateurs dans l'espace qubit;
3. exécuter l'algorithme quantique sur l'opérateur Hamiltonien qubit;
4. une fois le critère de convergence vérifié, évaluer les proprités physiques supplémentaires de l'état fondamental déterminé.

In [171]:
def run_vqe(problem, solver, mapper_name, optimizer_name="SLSQP", show=True):
    """Computing of the molecular ground state with the `GroundStateEigensolver`
class 
    Args:
        problem :  Electronic structure problem

        mapper_name : Mapper string that can be "PM" or "JWM" or "BKM"
        
        optimizeimizer_name :  optimizeimizer string that can be either "COBYLA" or "SLSQP" or "L_BFGS_B"

        solver :  Sting that can be either "NumPy" or "VQE"
       
    Returns:
          The ground state of the molecule

    """    
    # Mapper
    match mapper_name:
        case "JWM":
            mapper = JordanWignerMapper()
        case "PM":
            mapper = ParityMapper(num_particles=problem.num_particles)
        case "BKM":
            mapper = BravyiKitaevMapper()
    mapper = problem.get_tapered_mapper(mapper)

    if solver == "VQE":             
        # optimizer
        match optimizer_name:
            case "COBYLA": 
                optimizer = COBYLA()
            case "SLSQP": 
                optimizer = SLSQP()
            case "L_BFGS_B": 
                optimizer = L_BFGS_B()
        GS_solver = vqe_solver(problem, mapper, optimizer)

    # Solver
    if solver == "Numpy":
        GS_solver = NumPyMinimumEigensolver()

    # Ground state computation using a minimum eigensolver. Returns the solver.
    Algo = GroundStateEigensolver(mapper, GS_solver) 
    
    # Leveraging Qiskit Runtime
    start = time.time()
    
    # Compute Ground State properties.
    ground_state = Algo.solve(problem)
    elapsed = str(datetime.timedelta(seconds = time.time()- start))
    
    if show:
        print(f'Running {solver} and {mapper_name} mapper')
        if solver == "VQE":
            print(f'With {optimizer_name} optimizer and UCCSD ansatz')
        print(f'Elapsed time: {elapsed} \n')

        print(ground_state.total_energies[0])

    if solver == "VQE":
        return ground_state, elapsed, mapper_name, solver, optimizer_name
    else:
        return ground_state, elapsed, mapper_name, solver


* Resulat VQE

In [172]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
import time, datetime

res_vqe = run_vqe(H2_problem, mapper_name="PM", optimizer_name="SLSQP", solver="VQE")

Running VQE and PM mapper
With SLSQP optimizeimizer and UCCSD ansatz
Elapsed time: 0:00:00.110007 

-1.1373060356958726


- Résultat solveur exact NumPy.

In [173]:
res_np = run_vqe(H2_problem, mapper_name="PM", solver="Numpy")

Running Numpy and PM mapper
Elapsed time: 0:00:00.073481 

-1.137306035753399


* Nous calculons l'erreur relative entre les deux solvers avec le mapper "PM".

In [174]:
rel_error = (res_np[0].total_energies[0] - res_vqe[0].total_energies[0])*100/res_np[0].total_energies[0] 
print(f'\n The relative error between the two calculations is {rel_error:.4f}%')


 The relative error between the two calculations is 0.0000%


- Améliorons l'affichage des résultats pour des besoins de comparaison.

In [175]:
print(f"Type of solver  | GS electronic energy  |Elapsed time\t\t|Mapper | Optimizer | Rel. error  ")
print('================================================================================================')
print(f'{res_np[3]}\t\t| {res_np[0].total_energies[0]}   |{res_np[1]}\t\t|{res_np[2]} \t|\t    | ')
#print(f'Solver {name}   | {mapper_name}        |{optimizer}|{res_vqe.groundenergy}')
print(f'{res_vqe[3]}\t\t| {res_vqe[0].total_energies[0]}   |{res_vqe[1]}\t\t|{res_vqe[2]}\t|{res_vqe[4]}\t    |{rel_error:7.4f}%  ')
print('------------------------------------------------------------------------------------------------')


Type of solver  | GS electronic energy  |Elapsed time		|Mapper | Optimizer | Rel. error  
Numpy		| -1.137306035753399   |0:00:00.073481		|PM 	|	    | 
VQE		| -1.1373060356958726   |0:00:00.110007		|PM	|SLSQP	    | 0.0000%  
------------------------------------------------------------------------------------------------


On note que cette présentation n'est pas tout à fait satisfaisante.

### 2.2.4. <a id='toc2_2_4_'></a>[Visualisation de l'ensemble des résultats avec la bibliothèque `Pandas`](#toc0_)

Afin d'analyser les résultats en fonction des `mappers`et des `optimizers` nous allons utiliser la bibliothèque `Pandas`.

A cet effet, il nous vaudra créer un repertoire dans lequel placé le fichier où sera imprimer les résultats au format `csv` que `pandas` va par la suite lire et afficher.

In [176]:
# Creation of a simulation results folder
import os

cwd = os.getcwd()
directory = "Resultats"
targetPath = os.path.join(cwd, directory)

if not os.path.exists(targetPath):
    os.makedirs(targetPath)

In [177]:
Mapper = ["PM","JWM","BKM"]
Optimizer = ["COBYLA", "SLSQP","L_BFGS_B"]

In [178]:
# Create a file containing the results
QFile = os.path.join(targetPath, f"H2_results.csv")

In [179]:
q_result = open(QFile, "w")

q_result.write(f"Type of solver,Mapper,Optimizer,GS electr. energy,Rel. error (%),Elapsed time\n")

for i in Mapper:
    res_npH2 = run_vqe(H2_problem,  solver="Numpy", mapper_name=i, show=False)
    q_result.write(f'{res_npH2[3]},{res_npH2[2]},,{res_npH2[0].total_energies[0]},,{res_npH2[1]}\n')
    for j in Optimizer:
        res_vqeH2 = run_vqe(H2_problem, mapper_name=i, optimizer_name=j, solver="VQE", show=False)
        rel_errorH2 = (res_npH2[0].total_energies[0]- res_vqeH2[0].total_energies[0])*100/res_npH2[0].total_energies[0]
        q_result.write(f'{res_vqeH2[3]},{res_vqeH2[2]},{res_vqeH2[4]},{res_vqeH2[0].total_energies[0]},{rel_errorH2:7.4f},{res_vqeH2[1]}\n')
q_result.close()

In [180]:
import pandas as pd

H2_data = pd.read_csv(QFile)
H2_data

,Type of solver,Mapper,Optimizer,GS electr. energy,Rel. error (%),Elapsed time
0,Numpy,PM,NaN,-1.137306,NaN,0:00:00.070707
1,VQE,PM,COBYLA,-1.137306,0.0,0:00:00.144760
2,VQE,PM,SLSQP,-1.137306,0.0,0:00:00.098191
3,VQE,PM,L_BFGS_B,-1.137306,0.0,0:00:00.122151
4,Numpy,JWM,NaN,-1.137306,NaN,0:00:00.079715
5,VQE,JWM,COBYLA,-1.137306,0.0,0:00:00.152028
6,VQE,JWM,SLSQP,-1.137306,0.0,0:00:00.084689
7,VQE,JWM,L_BFGS_B,-1.137306,0.0,0:00:00.090757
8,Numpy,BKM,NaN,-1.137306,NaN,0:00:00.060856
9,VQE,BKM,COBYLA,-1.137306,0.0,0:00:00.150890


In [181]:
# Grouping solver by type and mapper
H2_group = H2_data.sort_values(by=['Type of solver', 'Mapper'])
H2_group

,Type of solver,Mapper,Optimizer,GS electr. energy,Rel. error (%),Elapsed time
8,Numpy,BKM,NaN,-1.137306,NaN,0:00:00.060856
4,Numpy,JWM,NaN,-1.137306,NaN,0:00:00.079715
0,Numpy,PM,NaN,-1.137306,NaN,0:00:00.070707
9,VQE,BKM,COBYLA,-1.137306,0.0,0:00:00.150890
10,VQE,BKM,SLSQP,-1.137306,0.0,0:00:00.083454
11,VQE,BKM,L_BFGS_B,-1.137306,0.0,0:00:00.085636
5,VQE,JWM,COBYLA,-1.137306,0.0,0:00:00.152028
6,VQE,JWM,SLSQP,-1.137306,0.0,0:00:00.084689
7,VQE,JWM,L_BFGS_B,-1.137306,0.0,0:00:00.090757
1,VQE,PM,COBYLA,-1.137306,0.0,0:00:00.144760


In [182]:
# Grouping solver
H2_dataNP = H2_group[0:int(H2_data.shape[0]/2)]
H2_dataVQE = H2_group[int(H2_data.shape[0]/2):int(H2_data.shape[0])]

In [183]:
# VQE UCCSD solver
H2_dataVQE

,Type of solver,Mapper,Optimizer,GS electr. energy,Rel. error (%),Elapsed time
5,VQE,JWM,COBYLA,-1.137306,0.0,0:00:00.152028
6,VQE,JWM,SLSQP,-1.137306,0.0,0:00:00.084689
7,VQE,JWM,L_BFGS_B,-1.137306,0.0,0:00:00.090757
1,VQE,PM,COBYLA,-1.137306,0.0,0:00:00.144760
2,VQE,PM,SLSQP,-1.137306,0.0,0:00:00.098191
3,VQE,PM,L_BFGS_B,-1.137306,0.0,0:00:00.122151


In [184]:
# Numpy exact solver
H2_dataNP

,Type of solver,Mapper,Optimizer,GS electr. energy,Rel. error (%),Elapsed time
8,Numpy,BKM,NaN,-1.137306,NaN,0:00:00.060856
4,Numpy,JWM,NaN,-1.137306,NaN,0:00:00.079715
0,Numpy,PM,NaN,-1.137306,NaN,0:00:00.070707
9,VQE,BKM,COBYLA,-1.137306,0.0,0:00:00.150890
10,VQE,BKM,SLSQP,-1.137306,0.0,0:00:00.083454
11,VQE,BKM,L_BFGS_B,-1.137306,0.0,0:00:00.085636


## 2.3. <a id='toc2_3_'></a>[Plugin `qiskit_nature_pyscf`](#toc0_)

Nous allons maintenant utiliser le plugin `qiskit_nature_pyscf` qui, nous le rappelons, couple PySCF et Qiskit Nature.  C'est un solveur [FCI](https://en.wikipedia.org/wiki/Full_configuration_interaction) (Full Configuration Interaction) basé sur Qiskit qui permet à un utilisateur de PySCF (Python-based Simulations of Chemistry Framework) de tirer parti des algorithmes quantique implémentés dans Qiskit pour être utilisés à la place de leurs homologues classiques (dans un esprit similaire à l'intégration NWChemEx).

 Nous allons nous intéresser aux molécules suivantes:
$$
\begin{array}{|l|l|l|l|}\hline
\text{Molécule} & \text{Qubits} & \text{Energie} (\rm{Hartree}) & \text{Distance} (\mathring{A})\\\hline
H_2 & 4 & -1.137306 & 0.735\\\hline
LiH & 12 & -7.882752 & 1.546\\\hline
H_2O & 14 & -75.023189 & 1.021\\\hline
\end{array}
$$


### Class `qiskit_nature_pyscf.PySCFGroundStateSolver`

La classe `qiskit_nature.second_q.algorithms.GroundStateSolver` s'appuie sur le module ``fci`` de PySCF. Il n'utilise aucun algorithmes quantiques (puisqu'il les remplace dans le workflow de Qiskit-Naure) mais fournit à la place un utilitaire pour déboguer les workflows de calcul classique basés sur Qiskit-Nature. 

Plus important encore, il fournit une implémentation plus efficace de ce que Qiskit-Nature réalise en utilisant la classe `qiskit_algorithms.NumPyMinimumEigensolver` en combinaison avec un ``filter_criterion``. Pour les états fondamentaux de spin autres que le singlet, l'utilisation des composants Qiskit-Nature est beaucoup plus complexe, alors que cette classe fournit une alternative facile à utiliser.



In [2]:
from pyscf import fci

from qiskit_nature.second_q.drivers import MethodType, PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from qiskit_nature_pyscf import PySCFGroundStateSolver

driver = PySCFDriver(
    atom="O 0.0 0.0 0.0; O 0.0 0.0 1.5",
    basis="sto3g",
    spin=2,
    method=MethodType.UHF,
)
O2_problem = driver.run()

transformer = ActiveSpaceTransformer(4, 4)

O2_problem = transformer.transform(O2_problem)

solver_fci = PySCFGroundStateSolver(fci.direct_uhf.FCI())

O2_result_fci = solver_fci.solve(O2_problem)
print(O2_result_fci)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -170.161901271469
  - computed part:      -5.105732353674
  - ActiveSpaceTransformer extracted energy part: -165.056168917795
~ Nuclear repulsion energy (Hartree): 22.57822766592
> Total ground state energy (Hartree): -147.583673605549
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: nan S^2: nan M: nan


In [10]:
O2_result_fci.computed_energies[0]

-5.10573235367409

In [185]:
from pyscf import gto, scf, mcscf
import numpy as np
from pyscf.mcscf import avas #AVAS method to construct mcscf active space
#active-space-size, active-electrons, orbital-initial-guess-for-CASCI/CASSCF

from qiskit_nature_pyscf import QiskitSolver

* Afin de ne pas interférer avec ce qui a été fait dans les sections précédentes, construisons à nouveau un algorithme quantique. Comme nous connaissons déja la procédure, nous n'allons plus la détailler.

In [186]:
from qiskit_nature.second_q.mappers import (
    ParityMapper, 
    JordanWignerMapper,
    BravyiKitaevMapper
)
from qiskit_nature.second_q.mappers import TaperedQubitMapper
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP, L_BFGS_B
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

Les optimisations classiques seront effectuées avec l'algorithme d'approximation stochastique de perturbation simultanée (SPSA), qui est une bonne approche pour les simulations en présence de bruit car elle ne nécessite que deux évaluations d'énergie par étape VQE, réduisant ainsi les coûts spplémentaires ou frais généraux.

In [187]:
def algorithm(mapper, optimizer, norb, nelec):
    
    # mapper = problem.get_tapered_mapper(mapper)
    
    ansatz = UCCSD(
    norb,
    nelec,
    mapper,
    initial_state=HartreeFock(
        norb,
        nelec,
        mapper,
    ),
    )

    vqe_solver = VQE(Estimator(), ansatz, optimizer)
    vqe_solver.initial_point = np.zeros(ansatz.num_parameters)
 
    algorithm = GroundStateEigensolver(mapper, vqe_solver)
    
    return algorithm

### 2.3.1. <a id='toc2_3_1_'></a>[Cas de la molécule d'hydrogène](#toc0_)

In [188]:
#  Initialisation de la structure moléculaire
H2_mol = gto.M(atom="H 0 0 0; H 0 0 .735", basis="sto-3g")
# Calculs HF 
H2_h_f = scf.RHF(H2_mol).run()

# Calculs post-HF
# norb = H2_h_f.mo_coeff.shape[1]
# nelec = H2_mol.nelec
# nel = nelec[0] + nelec[1]
norb, nel, mo = avas.avas(H2_h_f,['H 1s', 'H 1s'])
nelec = (int(nel/2), int(nel/2))
H2_cas = mcscf.CASCI(H2_h_f, norb, nel)

# Intégration de l'algorithme quantique
mapper = ParityMapper(nelec)
optimizer = SLSQP()
H2_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, norb, nelec))

H2_cas.run(mo)

converged SCF energy = -1.116998996754
CASCI E = -1.13730603569571  E(CI) = -1.85727503014469


* `CASCI E`est l'energie total évaluée, incluant l'énergie du coeur (inactive).  

In [205]:
H2_cas.e_tot

-1.1373060356957057

* `E(CI)` est l'énergie de reférence de `fcisolver`, c'est-à-dire la solution CI de Qiskit.

In [204]:
H2_cas.e_cas

-1.8572750301446854

### 2.3.2. <a id='toc2_3_2_'></a>[Cas de la molécule d'hydride de lithium](#toc0_)

<center><img src="Graphics/Lithium_hydride.png" width="150"/></center>



In [189]:
LiH_mol = gto.M(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")

LiH_h_f = scf.RHF(LiH_mol).run()

# Calculs post-HF
# norb = LiH_h_f.mo_coeff.shape[1]
# nelec = LiH_mol.nelec
# nel = nelec[0] + nelec[1]
norb, nel, mo = avas.avas(LiH_h_f,['Li 2s', 'H 1s'])
nelec = (int(nel/2), int(nel/2))
LiH_cas = mcscf.CASCI(LiH_h_f, norb, nel)

mapper = JordanWignerMapper()
optimizer = SLSQP()

LiH_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, norb, nelec))

LiH_cas.run(mo)

converged SCF energy = -7.86186476980865
CASCI E = -7.86900491410916  E(CI) = -1.10430532159457


In [190]:
LiH_cas = mcscf.CASSCF(LiH_h_f, norb, nel)
LiH_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, norb, nelec))
LiH_cas.run(mo)

CASSCF energy = -7.88104481532222
CASCI E = -7.88104481532222  E(CI) = -1.07696378563352


### 2.3.3. <a id='toc2_3_3_'></a>[Molecule d'eau](#toc0_)

<center><img src="Graphics/Water_structure1.png" width="150" /></center>

In [191]:
H2O_mol = gto.M(
  atom="O 0 0 0.115; H 0 0.754 -0.459; H 0 -0.754 -0.459",
  basis="sto6g",
)

H2O_h_f = scf.RHF(H2O_mol).run()

# Calculs post-HF
# norb = H2O_h_f.mo_coeff.shape[1]
# nelec = H2O_mol.nelec
# nel = nelec[0] + nelec[1]
# norb, nel, mo = avas.avas(H2O_h_f,['O 2p', 'H 1s', 'H 1s'])
# nelec = (int(nel/2), int(nel/2))
# H2O_cas = mcscf.CASCI(H2O_h_f, norb, nel)

H2O_cas = mcscf.CASCI(H2O_h_f, 2, 2)

mapper = BravyiKitaevMapper()
optimizer = SLSQP()

# H2O_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, 2, nelec))
H2O_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, 2, (1,1)))

H2O_cas.run()

converged SCF energy = -75.6769190377968
CASCI E = -75.6781758786882  E(CI) = -1.67690613522248


In [192]:
H2O_cas = mcscf.CASSCF(H2O_h_f, 2, 2)
H2O_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, 2, (1,1)))
H2O_cas.run()

CASSCF energy = -75.6781810340544
CASCI E = -75.6781810340544  E(CI) = -1.67758792172657


# 3. <a id='toc3_'></a>[Macro molécule](#toc0_)

Lors de l'**IBM Quantum Challenge Africa 2021**, un challenger sur la chimie quantique pour le VIH visait à déterminer si un modèle d'essai d'une molécule antirétrovirale peut se lier à un modèle d'essai d'une molécule de protéase. Étant donné que la molécule anti-rétrovirale a de nombreux atomes, elle est approximée en utilisant un seul atome de carbone. Le modèle d'essai de la molécule de protéase est représenté par un composant de la molécule de formamide (HCONH2) ; en particulier, c'est la partie **carbone-oxygène-azote** de la molécule de formamide. En bref, l'expérience consiste à déterminer si un seul atome de carbone peut se lier au composant carbone-oxygène-azote de la molécule de formamide. La reponse à la question posée par IBM peut être obtenu en traçant le BOPES (Born-Oppenheimer Potential Energy Surface) d'une macromolécule, qui est la molécule de formamide plus l'atome de carbone.

Pour visualiser cette macromolécule, décommenter la cellule suivante si vous avez installé `ase` (atomic simulation environment).
> pip install ase -U

In [193]:
# from ase import Atoms
# from ase.build import molecule
# from ase.visualize import view

# macro_ASE = Atoms('ONCHHHC', [(1.1280, 0.2091, 0.0000), 
#                           (-1.1878, 0.1791, 0.0000), 
#                           (0.0598, -0.3882, 0.0000),
#                           (-1.3085, 1.1864, 0.0001),
#                           (-2.0305, -0.3861, -0.0001),
#                           (-0.0014, -1.4883, -0.0001),
#                           (-0.1805, 1.3955, 0.0000)])

# view(macro_ASE, viewer='x3d')

In [194]:
M_mol = gto.M(atom='''
              O 1.128 0.2091 0.0; 
              N -1.1878 0.1791 0.0; 
              C 0.0598 -0.3882 0.0; 
              H -1.3085 1.1864 0.0001; 
              H -2.0305 -0.3861 -0.0001; 
              H -0.0014 -1.4883 -0.0001; 
              C -0.1805 1.3955 0.0
              ''',
              basis="sto-3g")
M_h_f = scf.RHF(M_mol).run()

# Calculs post-HF
# norb = M_h_f.mo_coeff.shape[1]
# nelec = M_mol.nelec
# nel = nelec[0] + nelec[1]
M_cas = mcscf.CASCI(M_h_f, 2, 2)

mapper = BravyiKitaevMapper()
optimizer = SLSQP()

M_cas.fcisolver = QiskitSolver(algorithm(mapper, optimizer, 2, (1,1)))

M_cas.run()

converged SCF energy = -203.543863584134
CASCI E = -203.545057922365  E(CI) = -0.885464741015937


In [195]:
# Display Qiskit Software and System information
import qiskit.tools.jupyter
%qiskit_version_table